# SuAVE Image to Label CNN Prediction Model

In [204]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 

<IPython.core.display.Javascript object>

### Import all packages (this might take a few seconds)

In [205]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Initializing the number of times the model will loop and its batch size for learning

In [199]:
epochCount = {
    '25 Iterations': 25,
    '50 Iterations': 50,
    '75 Iterations': 75
}

def f(epoch_count):
    return epoch_count

epochNum = interact(f, epoch_count=epochCount)

interactive(children=(Dropdown(description='epoch_count', options={'25 Iterations': 25, '50 Iterations': 50, '…

In [206]:
batchS = {
    '32 Batch Size': 32,
    '64 Batch Size': 64,
    '128 Batch Size': 128
}

def f(batch_size):
    return batch_size

batchNum = interact(f, batch_size=batchS)

interactive(children=(Dropdown(description='batch_size', options={'64 Batch Size': 64, '32 Batch Size': 32, '1…

In [207]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = epochNum.widget.result
INIT_LR = 1e-3
BS = batchNum.widget.result
# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

## Run and choose the column name that coresponds with the column label to be predicted

In [208]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
lower_case_csv = csv_file.lower()
img_path = "../../images/" + lower_case_csv.split(".")[0]

# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu
# choose which label for predictions
out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'Co ppm (INAA)#number': 'Co ppm (INAA)…

## Grab the images and configure them for predicting

### This might take a little while depending on the size of the dataset

In [209]:
# grab chosen column names
nameCol = df['#img']
predCol = df[out2.widget.result]

# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])
    
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join(img_path, base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

# Shuffle the data
p = np.random.permutation(len(yset))

# Relable for splitting sets
Y = []
X = []
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])
    
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

# parsed Y data containers
trainY = []
testY = []

# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,len(uni_labels))
testY = np_utils.to_categorical(ytest,len(uni_labels))

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")
# initialize the model
model = LeNet.build(width=28, height=28, depth=3, classes=len(uni_labels))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

## Train the predictive model

### This is relative to the size of the data set and may take a few minutes 

In [210]:
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

Epoch 1/75
4/4 [==============================] - 3s 745ms/step - loss: 2.8694 - acc: 0.1444 - val_loss: 2.8015 - val_acc: 0.2159
Epoch 2/75
4/4 [==============================] - 3s 710ms/step - loss: 2.7861 - acc: 0.2031 - val_loss: 2.8074 - val_acc: 0.1477
Epoch 3/75
4/4 [==============================] - 2s 586ms/step - loss: 2.5043 - acc: 0.1980 - val_loss: 3.1462 - val_acc: 0.1705
Epoch 4/75
4/4 [==============================] - 2s 558ms/step - loss: 2.8021 - acc: 0.2005 - val_loss: 2.7582 - val_acc: 0.2614
Epoch 5/75
4/4 [==============================] - 2s 583ms/step - loss: 2.7450 - acc: 0.1869 - val_loss: 2.7785 - val_acc: 0.1477
Epoch 6/75
4/4 [==============================] - 2s 561ms/step - loss: 2.6399 - acc: 0.1856 - val_loss: 2.9870 - val_acc: 0.1705
Epoch 7/75
4/4 [==============================] - 2s 585ms/step - loss: 2.4898 - acc: 0.2896 - val_loss: 3.0050 - val_acc: 0.1705
Epoch 8/75
4/4 [==============================] - 2s 540ms/step - loss: 2.6369 - acc: 0.20

4/4 [==============================] - 3s 698ms/step - loss: 1.8715 - acc: 0.4492 - val_loss: 2.9207 - val_acc: 0.2500
Epoch 64/75
4/4 [==============================] - 2s 411ms/step - loss: 1.7509 - acc: 0.4722 - val_loss: 3.2327 - val_acc: 0.2614
Epoch 65/75
4/4 [==============================] - 3s 631ms/step - loss: 1.8123 - acc: 0.4453 - val_loss: 3.3598 - val_acc: 0.2500
Epoch 66/75
4/4 [==============================] - 3s 677ms/step - loss: 1.7989 - acc: 0.4688 - val_loss: 3.0018 - val_acc: 0.2955
Epoch 67/75
4/4 [==============================] - 2s 563ms/step - loss: 1.9771 - acc: 0.3766 - val_loss: 3.0420 - val_acc: 0.2500
Epoch 68/75
4/4 [==============================] - 2s 535ms/step - loss: 1.7759 - acc: 0.4711 - val_loss: 2.8486 - val_acc: 0.2955
Epoch 69/75
4/4 [==============================] - 2s 548ms/step - loss: 1.8870 - acc: 0.3915 - val_loss: 3.8212 - val_acc: 0.2045
Epoch 70/75
4/4 [==============================] - 2s 570ms/step - loss: 1.9331 - acc: 0.3890 -

## Take the original data and predict based on the model

In [211]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

predictionsMade = []
preds = model.predict(img_check)

# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
    
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.3721590909090909


In [212]:
#Generate model file and save
modelName = user + "_cnn_" + out2.widget.result + "_" + str(epochNum.widget.result) + "_" + str(batchNum.widget.result) + ".h5"
modelPath = "models/"

model.save(os.path.join(modelPath, modelName))

In [213]:
#Load model
from keras.models import load_model
model2 = load_model(os.path.join(modelPath, modelName))

In [214]:
predictionsMade

[10,
 9,
 10,
 9,
 10,
 9,
 10,
 9,
 5,
 10,
 12,
 10,
 10,
 21,
 9,
 9,
 10,
 19,
 9,
 9,
 10,
 21,
 10,
 10,
 12,
 9,
 9,
 9,
 9,
 21,
 10,
 10,
 12,
 9,
 12,
 9,
 12,
 10,
 12,
 18,
 9,
 9,
 9,
 9,
 10,
 3,
 10,
 10,
 12,
 5,
 9,
 12,
 10,
 21,
 9,
 9,
 10,
 10,
 10,
 21,
 9,
 9,
 9,
 9,
 10,
 10,
 9,
 10,
 10,
 21,
 10,
 10,
 10,
 9,
 21,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 21,
 21,
 9,
 9,
 12,
 21,
 18,
 21,
 10,
 21,
 9,
 9,
 12,
 9,
 9,
 9,
 10,
 21,
 9,
 9,
 10,
 21,
 21,
 9,
 10,
 21,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 12,
 9,
 10,
 21,
 10,
 10,
 9,
 9,
 9,
 12,
 10,
 3,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 3,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 6,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 17,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 3,
 10,
 21,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 17,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 21,
 10,
 2

## Enter a new header for the prediction column

In [164]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value=None,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value=None,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

Input new column Header Label: 


Text(value='', placeholder='Type label here')

Text(value='', placeholder='New Header will be displayed here')

## Write the predictions back to the original CSV

In [132]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred
print(input_text.value)

# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)

kommos_j_test_1


In [130]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

survey_name = output_text.value

Input survey name here:


Text(value='')

Text(value='')

In [136]:
print(input_text.value)
print(dzc_file)


kommos_j_test_1
https://bioregional.ucsd.edu/suave/surveys/kommos_Oct2017/kommos_Oct2017.dzc


In [135]:

import requests
upload_url = "http://suave.sdsc.edu:3001/uploadCSV"
upload_data = {'name': input_text.value, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = "https://suave-dev.sdsc.edu/main/file=" + user + "_" + input_text.value + ".csv" + "&views=1110001&view=grid"
print(url)

200 OK
https://suave-dev.sdsc.edu/main/file=zaslavsk_.csv&views=1110001&view=grid
